In [3]:
%%html
<script src="https://cdn.jsdelivr.net/npm/konva@8/konva.min.js"></script>

In [4]:
#panel serve PythonJavascriptCommunications.ipynb --dev
import holoviews as hv; hv.extension('bokeh', 'plotly', logo=None)
import panel as pn;     pn.extension('katex')
from panel.custom import JSComponent
import param

#from panel.interact import interact
import numpy as np

# Interactive Exploration of vector decompositions and change of basis
class VectorDecomposition(JSComponent):
    # Parameters
    s1       = param.List(default=[1, 0], doc="First basis vector (dx, dy).")
    s2       = param.List(default=[0, 1], doc="Second basis vector (dx, dy).")
    b        = param.List(default=[3, 5], doc="Vector to decompose (dx, dy).")
    fixed    = param.Boolean(default=False, doc="Disallow moving endpoints of basis vectors.")
    vec_name = param.String(default="s", doc="Base name for the basis vectors (e.g., 's' or 'e').")
    offset   = param.List(default=[100, 100], doc="Offset from the bottom-left corner for the origin (dx, dy).")

    _importmap = {
        "imports": {  # does not seem to work: load in html cell instead!
            "konva": "https://cdn.jsdelivr.net/npm/konva@8/konva.min.js",
            "MJAX": "https://esm.sh/mathjax",
        }
    }

    _esm = r"""
export function render({ model, el }) {
  if (typeof window.Konva === 'undefined') {
    throw new Error('Konva is not loaded correctly!');
  }
  const Konva = window.Konva;

  // Create container div
  const container = document.createElement('div');
  container.style.width = '100%';
  container.style.height = '100%';
  el.appendChild(container);

  // Initialize Konva stage
  const stage = new Konva.Stage({
    container: container,
    width: 400,
    height: 400,
  });

  const offsetX = model.offset[0];
  const offsetY = model.offset[1];
  const margin = 50; // Margin for padding around the plot

  const originX = offsetX + margin;
  const originY = stage.height() - offsetY - margin;

  // Extract parameters
  const s1 = model.s1;
  const s2 = model.s2;
  const b = model.b;
  const fixed = model.fixed;

  // Check for vec_name existence
  const vec_name = model.vec_name || "s"; // Fallback to "s" if undefined

  // Calculate dynamic scaling factor
  const maxMagnitude = Math.max(
    Math.sqrt(b[0] ** 2 + b[1] ** 2),
    Math.sqrt(s1[0] ** 2 + s1[1] ** 2),
    Math.sqrt(s2[0] ** 2 + s2[1] ** 2)
  );
  const availableSpace = Math.min(stage.width() - 2 * margin, stage.height() - 2 * margin);
  const scale = availableSpace / (maxMagnitude * 2); // Scale down to fit comfortably

  // Layers
  const gridLayer = new Konva.Layer();
  const vectorLayer = new Konva.Layer();
  const controlLayer = new Konva.Layer();
  stage.add(gridLayer, vectorLayer, controlLayer);

  // ============================================== Helper Functions
  function drawGrid() {
    gridLayer.find('Line').forEach(line => line.remove()); // Clear grid

    const gridSize = 10; // Number of steps in the grid in each direction

    for (let i = -gridSize; i <= gridSize; i++) {
      for (let j = -gridSize; j <= gridSize; j++) {
        const x = originX + i * s1[0] * scale + j * s2[0] * scale;
        const y = originY - (i * s1[1] * scale + j * s2[1] * scale);

        const parallelToS1 = new Konva.Line({
          points: [x, y, x + s1[0] * scale, y - s1[1] * scale],
          stroke: '#ddd',
          strokeWidth: 1,
        });

        const parallelToS2 = new Konva.Line({
          points: [x, y, x + s2[0] * scale, y - s2[1] * scale],
          stroke: '#ddd',
          strokeWidth: 1,
        });

        gridLayer.add(parallelToS1);
        gridLayer.add(parallelToS2);
      }
    }

    gridLayer.draw();
  }
  // ========================================================================
  function addLabel(layer, text, x, y, color = 'black') {
    const label = new Konva.Text({
      text: text,
      x: x + 10, // Offset slightly to avoid overlap
      y: y - 10,
      fontSize: 16,
      fontFamily: 'Arial',
      fill: color,
    });
    layer.add(label);
  }
  // ========================================================================
  function drawVectors() {
    vectorLayer.find('Arrow, Line, Text').forEach(el => el.remove()); // Clear previous vectors and labels

    const s1EndX = originX + s1[0] * scale;
    const s1EndY = originY - s1[1] * scale;
    const s2EndX = originX + s2[0] * scale;
    const s2EndY = originY - s2[1] * scale;

    const c = "#3B3651";

    const basisVector1 = new Konva.Arrow({
      points: [originX, originY, s1EndX, s1EndY],
      pointerLength: 10,
      pointerWidth: 10,
      fill: c,
      stroke: c,
      strokeWidth: 3,
    });

    const basisVector2 = new Konva.Arrow({
      points: [originX, originY, s2EndX, s2EndY],
      pointerLength: 10,
      pointerWidth: 10,
      fill: c,
      stroke: c,
      strokeWidth: 3,
    });

    addLabel(vectorLayer, `${vec_name}_1`, s1EndX, s1EndY, 'black');
    addLabel(vectorLayer, `${vec_name}_2`, s2EndX, s2EndY, 'black');

  // Target vector (b)
  const bEndX = originX + b[0] * scale;
  const bEndY = originY - b[1] * scale;

  const targetVector = new Konva.Arrow({
    points: [originX, originY, bEndX, bEndY],
    pointerLength: 10,
    pointerWidth: 10,
    fill: 'blue',
    stroke: 'blue',
    strokeWidth: 3,
  });


  // Add label for b
  addLabel(vectorLayer, 'b', bEndX, bEndY, 'blue');


    const det = s1[0] * s2[1] - s1[1] * s2[0];
    const alpha = (b[0] * s2[1] - b[1] * s2[0]) / det;
    const beta = (b[1] * s1[0] - b[0] * s1[1]) / det;

    const alphaX = originX + alpha * s1[0] * scale;
    const alphaY = originY - alpha * s1[1] * scale;
    const betaX = originX + beta * s2[0] * scale;
    const betaY = originY - beta * s2[1] * scale;

    const alphaVector = new Konva.Arrow({
      points: [originX, originY, alphaX, alphaY],
      pointerLength: 10,
      pointerWidth: 10,
      fill: 'orange',
      stroke: 'orange',
      strokeWidth: 2,
    });

    const betaVector = new Konva.Arrow({
      points: [originX, originY, betaX, betaY],
      pointerLength: 10,
      pointerWidth: 10,
      fill: 'orange',
      stroke: 'orange',
      strokeWidth: 2,
    });

    const alphaToB = new Konva.Line({
      points: [alphaX, alphaY, bEndX, bEndY],
      stroke: 'orange',
      strokeWidth: 2,
      dash: [10, 5],
    });

    const betaToB = new Konva.Line({
      points: [betaX, betaY, bEndX, bEndY],
      stroke: 'orange',
      strokeWidth: 2,
      dash: [10, 5],
    });

    vectorLayer.add(alphaToB, betaToB, alphaVector, betaVector, basisVector1, basisVector2, targetVector);
    vectorLayer.add(basisVector1, basisVector2);
    vectorLayer.add(targetVector);

    addLabel(vectorLayer, `${vec_name}_1`, s1EndX, s1EndY, 'black');
    addLabel(vectorLayer, `${vec_name}_2`, s2EndX, s2EndY, 'black');
    addLabel(vectorLayer, 'b', bEndX, bEndY, 'blue');
    //addLabel(vectorLayer, `α=${alpha.toFixed(2)}`, alphaX, alphaY, 'orange');
    //addLabel(vectorLayer, `β=${beta.toFixed(2)}`, betaX, betaY, 'orange');

    vectorLayer.draw();
  }
  // ========================================================================

  function drawControlPoints() {
    controlLayer.find('Circle').forEach(el => el.remove());
    if (fixed) return;
    const s1Control = new Konva.Circle({
      x: originX + s1[0] * scale,
      y: originY - s1[1] * scale,
      radius: 6,
      fill: null,
      stroke: 'lightgray',
      draggable: !fixed,
    });

    const s2Control = new Konva.Circle({
      x: originX + s2[0] * scale,
      y: originY - s2[1] * scale,
      radius: 6,
      fill: null,
      stroke: 'lightgray',
      draggable: !fixed,
    });

    controlLayer.add(s1Control, s2Control);

    if (!fixed) {
      s1Control.on('dragmove', () => {
        s1[0] = (s1Control.x() - originX) / scale;
        s1[1] = (originY - s1Control.y()) / scale;
        gridLayer.destroyChildren();
        drawGrid();
        drawVectors();
      });

      s2Control.on('dragmove', () => {
        s2[0] = (s2Control.x() - originX) / scale;
        s2[1] = (originY - s2Control.y()) / scale;
        gridLayer.destroyChildren();
        drawGrid();
        drawVectors();
      });
    }

    controlLayer.draw();
  }
  // ========================================================================
  // Initial draw
  drawGrid();
  drawVectors();
  drawControlPoints();
}
"""

<div style="float:center;width:100%;text-align: center;">
    <strong style="height:60px;color:darkred;font-size:40px;">Change of Basis and the Similarity Transform</strong><br>
</div>

# 1. Change of Coordinates: Decomposition of a Vector

In linear algebra, **change of coordinates** refers to expressing a vector in terms of a new set of **basis vectors.**

**Key Concepts:**
* A basis is a set of **linearly independent** vectors that span a vector space.
* **Any vector in this space can be uniquely represented** as a linear combination of the basis vectors.

Suppose we have a vector $b$ and a basis formed by vectors $e_1$ and $e_2$.<br>
To express $b$ in terms of $e_1$ and $e_2$, we find scalars $\alpha_1$ and $\alpha_2$ such that:

$\qquad
\mathbf{b} = \alpha_1 e_1 + \alpha_2 e_2
$

This process is called **Vector Decomposition:**<br><br>
$\qquad$ The vector $\begin{pmatrix} \alpha_1 \\ \alpha_2 \end{pmatrix}$ is the **coordinate vector** of $b$ relative to the basis $\left\{\ e_1, e_2 \ \right\}$

Consider the following example:

In [91]:
sample_vector = [3, 5]
ex1 = VectorDecomposition( s1=[1, 0], s2=[0, 1], b=sample_vector, offset=[50, 50], fixed=True, vec_name="e")

pn.Row( pn.Column(
    "## Example Vector Decomposition",
    pn.pane.LaTeX(r"$\large{\quad b = 3 e_1 + 5 e_2}$"),
    ex1),
    pn.Spacer(width=30),
    pn.Column(pn.Spacer(height=140),
    pn.pane.Markdown("## Computation:  an A x = b type problem!"),
    pn.pane.LaTeX(r"$\large{\qquad b = \alpha_1 e_1 + \alpha_2 e_2} \Leftrightarrow \begin{pmatrix}e_1 & e_2\end{pmatrix} \begin{pmatrix}\alpha_1 \\ \alpha_2 \end{pmatrix} = b$"),
    pn.Spacer(height=30),
    "### Note: if e₁ and e₂ are orthonormal, we just need to compute dot products:""",
    pn.pane.LaTeX(r"$\qquad \quad\large{\begin{aligned} \alpha_1 &= b \cdot e_1,\\ \alpha_2 &= b \cdot e_2 \end{aligned}}$"),
    ),
).servable()

Row
    [0] Column
        [0] Markdown(str)
        [1] LaTeX(str, renderer='katex')
        [2] VectorDecomposition(fixed=True, offset=[50, 50], vec_name='e')
    [1] Spacer(width=30)
    [2] Column
        [0] Spacer(height=140)
        [1] Markdown(str)
        [2] LaTeX(str, renderer='katex')
        [3] Spacer(height=30)
        [4] Markdown(str)
        [5] LaTeX(str, renderer='katex')

____
**There is no reason for the vectors to be orthogonal or unit length**

In the following example, we decompose the vector with respect to two bases:<br>
$\qquad b = \alpha_1 e_1 + \alpha_2 e_2,\;\;$ and $\;\;b = \beta_1 s_1 + \beta_2 s_2$.

* You can modify $s_1$ and $s_2$ by dragging their endpoints.
  * **Investigate:** *what happens when $s_1$ and $s_2$ fall on the same line?*<br>
(this is the case where $s_1$ and $s_2$ are no longer linearly independent,<br>
and hence no longer form a basis!)
* The grid lines pass through integer multiple endpoints of the basis vectors.

In [7]:
s5=np.sqrt(5.)
sample_vector=[4,7]

ex2a = VectorDecomposition(name="Decomposition 1", s1=[1, 0], s2=[0, 2],
                                   b=sample_vector,
                                   offset=[50, 50],
                                   fixed = True,
                          vec_name="e")
ex2b = VectorDecomposition(name="Decomposition 2", s1=[2/s5, 1/s5], s2=[0, 3],
                                   b=sample_vector, offset=[50, 50],
                                   fixed = False,
                                   vec_name="s")

# Layout the Panel app
layout = pn.Column(
    pn.pane.Markdown("# Decomposition of a Vector with Respect to Two Bases"),
    pn.Row(ex2a, pn.Spacer(width=30), ex2b),
)

# Serve the app
layout.servable()

Column
    [0] Markdown(str)
    [1] Row
        [0] VectorDecomposition(b=[4, 7], fixed=True, name='Decomposition 1', offset=[50, 50], s2=[0, 2], vec_name='e')
        [1] Spacer(width=30)
        [2] VectorDecomposition(b=[4, 7], name='Decomposition 2', offset=[50, 50], s1=[np.float64(0.894427190999...], s2=[0, 3])

### Key Concepts:

1. **Linear Independence**: The basis vectors **cannot** lie on the same line (they must be linearly independent).
2. **Unique Representation**: Each vector has a unique decomposition in terms of the basis vectors.
3. **Matrix Representation**: We can represent the decomposition as a matrix equation:

$\qquad
\begin{pmatrix}
s_{1x} & s_{2x} \\
s_{1y} & s_{2y}
\end{pmatrix}
\begin{pmatrix}
\alpha \\
\beta
\end{pmatrix}
=
\begin{pmatrix}
b_x \\
b_y
\end{pmatrix}
$

Here, the matrix formed by $\mathbf{s}_1$ and $\mathbf{s}_2$ is called the **basis matrix**, and solving the equation gives the coordinates $\alpha$ and $\beta$.

#### Example Computation

 Obtain $\alpha_1$ and $\alpha_2$ such that $\;\; b = \alpha_1 s_1 + \alpha_2 s_2\;\;$ where

 $\qquad b = \begin{pmatrix} 9 \\ 2 \end{pmatrix},\;\;$  $s_1=\begin{pmatrix} 3\\ 1 \end{pmatrix},\;\;$ and $\;\;s_2 = \begin{pmatrix} 5 \\ 2 \end{pmatrix}.\qquad$

**Remarks:**
* The vectors $b, s_1,$ and $s_2\;\;$ are expressed with respect to a current coordinate system.
* the given vectors of coefficients are **coordinate vectors**

We have

$\qquad b = \alpha_1 s_1 + \alpha_2 s_2 = \begin{pmatrix} s_1 & s_2 \end{pmatrix} \begin{pmatrix}\alpha_1 \\ \alpha_2\end{pmatrix}$

Substituting, we obtain **the coordinate vector of $b$ with respect to the coordinate system (i.e., basis) $s_1, s_2.$**

$\qquad \begin{pmatrix} 3 & 5 \\ 1 & 2 \end{pmatrix} \begin{pmatrix} \alpha_1 \\ \alpha_2 \end{pmatrix} = \begin{pmatrix} 9 \\ 2\end{pmatrix}
\;\;\Leftrightarrow \;\;  \begin{pmatrix} \alpha_1 \\ \alpha_2 \end{pmatrix} =  \left(\begin{array}{rr} 8  \\ -3 \end{array}\right)$

____
**IMPORTANT REMARKS:** we have two representations of the vector $b$:
* $b = \begin{pmatrix}\; 9 \\ \; 2 \end{pmatrix},\;\;$ the coordinate vector with respect to a current coordinate system $e_1, e_2:\;\; b= 9 e_1 + 2 e_2$
* $\tilde{b} = \left(\begin{array}{r} 8 \\ -3 \end{array}\right),\;\;$ the coordinate vector with respect to a new coordinate system $s_1, s_2:\;\; b=8 s_1 -3 s_2$.

* In the computation above, the new coordinate system is given by coordinate vectors $s_1$ and $s_2$<br>
**with respect to the current coordinate system** $e_1, e_2$, i.e., the standard basis.

* If the actual basis is not specified, the coordinate vector $\tilde{b}$ can be drawn using the **standard basis** (the columns of $I$)

* **This decomposition generalizes** to higher dimensions: $\;\;b = \begin{pmatrix} s_1 & s_2 & \dots & s_n \end{pmatrix}\ \tilde{b}$,<br>

* The change of basis equation is given by $\;\;\Large{\color{blue}{\boxed{b = S\ \tilde{b}\;\; \Leftrightarrow \;\; \tilde{b} = S^{-1}\ b}}},$
    - $b$ is the vector in the standard basis.
    - $\tilde{b}$ is the coordinate vector of $b$ in the new basis.
    - $S$ is the change-of-basis matrix, whose columns are the new basis vectors $\{s_1, s_2, \ldots, s_n\}$.
<br><br>

* Note that the matrix $S = \begin{pmatrix} s_1 & s_2 & \dots & s_n \end{pmatrix}$ has linearly independent columns:<br>
$\qquad$ **it is invertible**


# 2. Similarity Transforms

## 2.1 Linear Transforms and Coordinate Systems

Next, consider a linear transformation $y = A x$, where $A$ is an $N\times N$ matrix, i.e., $\;\; x\in \mathbb{R}^N \xrightarrow{y=A x} y\in \mathbb{R}^N$.

Expressing $x$ in the standard basis $x = x_1 e_1 + x_2 e_2 + \dots x_n e_n$, we can interpret this transformation as<br>
$\qquad$ a replacement of the basis vectors $e_i \rightarrow A e_i$, since $A x = x_1 A e_1 + x_2 A e_2 + \dots + x_n A e_n$.

**Example:** Let $A = \begin{pmatrix} 3 & 2 \\ 1 & 4 \end{pmatrix}$, so that for the standard basis $e_1, e_2$, we have
$\;\; A e_1 = \begin{pmatrix} 3  \\ 1 \end{pmatrix}\;\;$ and $\;\; A e_2 = \begin{pmatrix} 2 \\ 4 \end{pmatrix}$.

$\qquad x = x_1 \begin{pmatrix}1\\0\end{pmatrix}+x_2\begin{pmatrix}0\\1\end{pmatrix}
\xrightarrow{y=A x} y = x_1 \begin{pmatrix}3\\1\end{pmatrix}+x_2\begin{pmatrix}2\\4\end{pmatrix}$

This raises a question: what if we want to change the coordinate systems we use for $x$ and $y$?

Since both $x$ and $y$ have the same number of entries, we can use the same new coordinate system for both, i.e.,<br>
$\qquad x = S \tilde{x},\;$ and $\; y = S \tilde{y}$.

$\qquad\begin{align}
y = A x & \; \Leftrightarrow & S \tilde{y} = A S\ \tilde{x} & \\
        & \; \Leftrightarrow & \tilde{y} = S^{-1} A S\ \tilde{x} &
\end{align}$


This has two interesting consequences:
* we can transform coordinate vectors directly:  $\;\;\tilde{x} \xrightarrow{\tilde{y} = \tilde{A} \tilde{x}} \tilde{y}$
* the matrices $A$ and $\tilde{A} = S^{-1} A S$ **have the same geometric interpretation,**<br>
but are expressed in different coordinate systems: $\{ e_1, e_2, \dots \}$ and $\{ s_1, s_2, \dots \}$.

<div style="float:left;width:100%;background-color:#F2F5A9;color:black;">

**Definition:** A **similarity transformation** for a square matrix $A \in \mathbb{R}^{n \times n}$ is defined as

$\qquad \tilde{A} = S^{-1} A S$

where:
- $S$ is an invertible matrix of size $n \times n$,
- $\tilde{A}$ is the transformed matrix, also of size $n \times n$.
</div>

## 2.2 Example: a 3D Rotation

In [30]:
# Create a Panel app to embed the video
video_pane = pn.pane.Video("Figs/3DRotation.mp4", loop=False, autoplay=False, width=640, height=360)

# Build the Panel layout
layout = pn.Column(
    "## Rotation Around the Z-Axis",
    "### Two Sets of Axes: The Rotation Matrix with respect to the blue axes is simple!",
    video_pane,
)

# Serve the Panel app
layout.servable()

Column
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Video(str, height=360, sizing_mode='fixed', width=640)

In this move, there are two coordinate systems, one red and one blue. The vector $v$ rotates by an angle $\theta$ in the blue palne.<br>
The matrix for this 3D rotation around the blue z-axis (pointing up) is easy to write down:<br>

$\qquad \tilde{R}_{\theta} = \begin{pmatrix} \cos\theta & -\sin\theta & 0 \\ \sin\theta & \cos\theta & 0 \\ 0 & 0 & 1 \end{pmatrix}.\;\;$

For example, **in the blue coordinate system,** the vector $\;\;v = \begin{pmatrix} 2 \\ 0 \\0 \end{pmatrix}$ gets rotated to $\;\;\tilde{R}_\theta\ v = 2 \begin{pmatrix} \cos\theta \\ \sin\theta \\ 0 \end{pmatrix}.$

What might the rotation matrix be with respect to the red axes?

Let's fix the nomenclature:
- the blue axes are $s_1, s_2$ and $s_3$. The rotation matrix with respect to these axes is $\tilde{R}_\theta$
- the red axes are $e_1, e_2, e_3$. The rotation matrix with respect to these axes is $R_\theta$.

We have to specify the $s_1, s_2$ and $s_3$ axes with respect to the $e_1, e_2$ and $e_3$ axes.<br>
$\qquad$ Let's assume they are unit length and mutually orthogonal (**an orthonormal basis**), e.g.,<br>
$\qquad\qquad s_1 = \frac{1}{\sqrt{101}}\begin{pmatrix} 10\\ 0\\ -1 \end{pmatrix},\; s_2 = \begin{pmatrix} 0 \\ 1 \\ 0\end{pmatrix},\; s_3= \frac{1}{\sqrt{101}}\begin{pmatrix}1\\0\\ 10 \end{pmatrix}$.<br>
$\qquad$ These are the coordinate vectors with respect to the $e_1, e_2, e_3$ axes, so $\; s_1 = \frac{1}{\sqrt{101}} \left( 10 e_1 + 0 e_2 - e_3 \right),$ etc.

Since $\tilde{R}_\theta = S^{-1} R_\theta\ S,\;\;$ we have $R_\theta = S \tilde{R} S^{-1} = \begin{pmatrix} 10 & 0 & 1 \\ 0 & \sqrt{101} & 0 \\ -1 & 0 & 10 \end{pmatrix} \begin{pmatrix} \cos\theta & -\sin\theta & 0 \\ \sin\theta & \cos\theta & 0 \\ 0 & 0 & 1 \end{pmatrix} \begin{pmatrix} 10 & 0 & 1 \\ 0 & \sqrt{101} & 0 \\ -1 & 0 & 10 \end{pmatrix}^{-1}
$<br>

$\qquad$ a rather messy rotation matrix with respect to the red axes! (Note we canceled out the terms involving $\sqrt{101}$).

$\qquad$ Since the vectors were chosen orthonormal, we further have $S^{-1} = S^t$.
Multiplying out the matrices, we get

$\qquad R_\theta = \frac{1}{101}
\left( \begin{array}{ccc}
100 \cos\theta + 1 & - 10 \sqrt{101} \sin\theta & 10 - 10 \cos\theta \\
10 \sqrt{101} \sin\theta & 101 \cos\theta & - \sqrt{101} \sin\theta \\
10 - 10 \cos\theta & \sqrt{101}\sin\theta & \cos\theta + 100
\end{array}\right)$

**We could recreate the above movie by making $s_1, s_2$ and $s_3$ functions of time.**

# 3. Generalization of the Similarity Transform

The similarity transform is traditionally defined for square matrices $A \in \mathbb{R}^{n \times n}$,<br>
where the transformation involves the same change of basis for both the domain and the codomain of $y = A x$<br>
$\qquad x = S\ \tilde{x},\;\;$ and $\;\;y = S\ \tilde{y}\;\;$ for an invertible matrix $S$.

We may choose to use different bases in the domain and codomain: $\;\;x = V \tilde{x}, \;\; y = U \tilde{y},$<br>
$\qquad$ with invertible change of basis matrices $U$ and $V$.

**Remark:** This removes the requirement that $A$ be a square matrix!

The transformation $\large{\boxed{\tilde{A} = U^{-1} A V}}\;\;$
expresses the same geometric transformation with respect to these new coordinate systems.

## 3.1 Example

Consider $\;\;A = \begin{pmatrix}
34 & 118 & 104 & 88 \\
52 & -130 & -32 & -152 \\
-180 & -159 & -264 & -36
\end{pmatrix}$

And choose $\;\; U = \begin{pmatrix}
-2 & 3 & 6 \\
6 & -2 & 3 \\
-3 & -6 & 2
\end{pmatrix},\;\;$ and $\;\;V = \begin{pmatrix}
16 & -11 & 8 & -20 \\
11 & 16 & 20 & 8 \\
8 & -20 & 4 & 19 \\
-20 & -8 & 19 & -4
\end{pmatrix}$.


We obtain $\tilde{A} = U^{-1} A V = 841 \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 2 & 0 & 0 \\
0 & 0 & 0 & 0
\end{pmatrix}$

**Choosing these coordinate systems resulted in a similar matrix $\tilde{A}$ that is particularly simple!**

# 4. Take Away

* The substitution $x = S \tilde{x}$, where $S = \begin{pmatrix} s_1 & s_2 & \dots & s_n}$ is invertible<br>
implements a change of basis from the standard basis $\{ e_1, e_2, \dots e_n \}$ to the basis $\{ s_1, s_2, \dots s_n \}$
